# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [31]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [32]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5'
openai = OpenAI()

API key looks good so far


In [33]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [34]:
ed = Website("https://edwarddonner.com")
ed.get_contents()

'Webpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,\nacquired in 2021\n.\nWe work with groundbreaking, proprietary LLMs verticalized for talent, we’ve\npatented\nour matching model, and our award-winning platfor

In [35]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [36]:
# One-shot prompting: System Prompt에 예시를 함께 줌
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [37]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [38]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    # \n을 구분자로 하여 하나의 문자열로 합침.
    user_prompt += "\n".join(website.links)
    return user_prompt

In [39]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [40]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        # openAI의 대답 형식을 지정할 수 있다.
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [41]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/tencent/Hunyuan-MT-7B',
 '/openbmb/MiniCPM-V-4_5',
 '/meituan-longcat/LongCat-Flash-Chat',
 '/tencent/HunyuanWorld-Voyager',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/apple/fastvlm-webgpu',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/bytedance-research/USO',
 '/spaces/Arial311/DeepSite-Gallery',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/data-agents/jupyter-agent-dataset',
 '/datasets/openai/healthbench',
 '/datasets/facebook/recycling_the_web',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',


In [24]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [42]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    # GPT에서 관련 Link를 물어봄
    links = get_links(url)
    print("Found links:", links)
    # 각 Link에 해당되는 Website 객체의 내용을 Return
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [43]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face


In [44]:
# Brochure를 생성하기 위해 두 번째 Prompt를 작성

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [45]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    # 첫 번째 GPT Model을 통해 얻은 특정 Website에 대해 관련있는 Link의 Content를 전부 넘겨줌.
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [46]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\n3 days ago\n•\n150k\n•\n1.36k\ntencent/Hunyuan-MT-7B\nUpdated\nabout 18 hours ago\n•\n1.4k\n•\n433\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 4 hours ago\n•\n16.2k\n•\n850\nmeituan-longcat/LongCat-Flash-Chat\nUpdated\n4 days ago\n•\n10.3k\n•\n382\ntencent/HunyuanWorld-Voyager\nUpdated\n2 days ago\n•\n89\n•\n334\nBrowse 1M+ models\nSpaces\nRunning\n12.7k\n12.7

In [47]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [48]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face
The AI community building the future.

Hugging Face is the platform where the machine learning community collaborates on models, datasets, and applications—“The Home of Machine Learning.” Create, discover, and collaborate on ML across text, image, video, audio, and even 3D.

## What you can do
- Host and collaborate on unlimited public models, datasets, and applications on the Hugging Face Hub.
- Explore over 1M+ models, 250k+ datasets, and 400k+ applications (Spaces).
- Build your portfolio: share your work with the world and grow your ML profile.
- Move faster with the Hugging Face open-source stack.

## Products and solutions
- Compute
  - Deploy on optimized Inference Endpoints or scale Spaces apps with GPUs in a few clicks.
  - Pricing: starting at $0.60/hour for GPU.
- Team & Enterprise
  - Build with enterprise-grade security, access controls, and dedicated support.
  - Features include Single Sign-On, Regions, Priority Support, Audit Logs, Resource Groups, and Private Datasets Viewer.
  - Pricing: starting at $20/user/month.

## Our open-source foundation
We’re building the foundation of ML tooling with the community:
- Transformers — State-of-the-art AI models for PyTorch
- Diffusers — State-of-the-art diffusion models in PyTorch
- Safetensors — Safe storage/distribution for neural network weights
- Hub Python Library — Client for the Hugging Face Hub
- Tokenizers — Fast tokenizers for research & production
- TRL — Train transformer LMs with reinforcement learning
- Transformers.js — Run state-of-the-art ML in the browser
- smolagents — Lightweight library to build agents in Python
- PEFT — Parameter-efficient finetuning for LLMs
- Datasets — Access and share datasets for any ML task
- Text Generation Inference — Optimized toolkit for serving LMs
- Accelerate — Train PyTorch models with multi-GPU/TPU/mixed precision

## Who uses Hugging Face
More than 50,000 organizations use Hugging Face, including:
- AI2 (Allen Institute for AI)
- AI at Meta
- Amazon
- Google
- Intel
- Microsoft
- Grammarly
- Writer

## Culture and community
- Community-first and open-source driven: collaborate publicly on models, datasets, and apps.
- Build across all modalities and share learnings through docs, blog, and forums.
- A place to learn, contribute, and showcase your work alongside the global ML community.

## Careers
Join the team building the future of AI with the community. Explore roles on the Hugging Face Jobs page.

## Get started
- Sign Up to create your profile and start collaborating.
- Explore AI Apps or browse 1M+ models and 250k+ datasets.
- Check out Documentation, Blog, and Forum for learning and support.
- Enterprise? Get started with advanced platform features, SSO, and dedicated support.
- Stay connected: GitHub, Twitter, LinkedIn, Discord.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

Welcome to **Hugging Face** – The AI community building the future!

## About Us
At Hugging Face, we are dedicated to fostering innovation within the artificial intelligence (AI) and machine learning (ML) realms. Our platform is a collaborative space where developers from around the globe can create, share, and utilize state-of-the-art AI models, datasets, and applications.

## Our Offerings

### Models and Datasets
- **1M+ Models:** Explore a diverse range of AI models, including:
    - **VibeVoice-1.5B** (by Microsoft)
    - **MiniCPM-V-4_5** (by openbmb)
    - **LongCat-Flash-Chat** (by meituan-longcat)
- **250k+ Datasets**: Access a variety of datasets tailored for numerous machine learning tasks.

### Spaces
Hugging Face hosts **12.6k Spaces** where users can run applications for real-time capabilities like video captioning, image generation, and more.

### Enterprise Solutions
We offer tailored **Compute and Enterprise solutions** starting at $0.60/hour for GPU access and $20/user/month for teams. These solutions are designed to empower organizations by providing enterprise-grade security, resources, and support. 

## Our Community
More than **50,000 organizations** are using Hugging Face, including industry leaders like Google, Microsoft, Amazon, and Grammarly. With a dynamic community of over 57,000 members, we empower collaboration, sharing of knowledge, and rapid advancements in ML technology.

## Our Culture
At Hugging Face, we celebrate collaboration and community engagement as the cornerstone of our work culture. We strongly advocate for open-source principles and aim to build a supportive environment where individuals can grow professionally and contribute to transformative projects. We're not just a tech company; we're a people-centric organization where creativity, curiosity, and passion for AI thrive.

## Careers at Hugging Face
Join our diverse and talented team! We are constantly looking for motivated individuals to help shape the future of machine learning. Whether you're an engineer, researcher, or business strategist, there’s a place for you. Explore our [Careers Page](https://huggingface.co/jobs) to find exciting job opportunities.

## Connect with Us
Stay in touch and be part of our journey:
- Follow us on **[Twitter](https://twitter.com/huggingface)**, **[LinkedIn](https://linkedin.com/company/huggingface)**, and **[Discord](https://discord.gg/huggingface)**.
- Join our [Community Forum](https://huggingface.co/forum) for discussions, collaborations, and queries.

---

**Hugging Face: Where AI Meets Community.**  
**Explore. Collaborate. Innovate.**  
[Visit our website](https://huggingface.co) for more information!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of the AI revolution, spearheading a community-driven platform that empowers collaboration in the fields of machine learning and artificial intelligence. With over **1 million models** and **250,000 datasets**, our platform is a hub for innovators, developers, and researchers looking to create cutting-edge AI tools and applications. We believe in building the future of AI together.

## Our Offerings
- **Models**: Access over 1 million AI models, including the latest advancements in text, image, video, and audio processing.
- **Datasets**: Explore and share more than 250,000 datasets for various ML tasks to accelerate your projects.
- **Spaces**: Host and collaborate on applications utilizing our Spaces feature which currently supports thousands of applications and tools.

## Customers
Join over **50,000 organizations** that trust Hugging Face, including:
- **Tech Giants**: Amazon, Google, Microsoft
- **Research Institutions**: AI2, Meta
- **Innovative Startups**: Grammarly, Writer

## Company Culture
At Hugging Face, we pride ourselves on fostering a positive, inclusive, and collaborative environment. Our team of passionate professionals values diversity and community engagement, bringing together individuals who share a common goal: to democratize AI. 

Our work culture encourages continuous learning and growth, allowing team members to thrive while pushing the boundaries of AI technology.

## Careers
Join our rapidly growing team and help shape the future of machine learning. We are looking for innovators across various disciplines, whether you're a developer, researcher, marketer, or business strategist.

**Current Opportunities**: 
- Software Engineers
- Machine Learning Researchers
- Data Scientists
- Community Managers

Check our [Jobs Page](https://huggingface.co/jobs) for the latest openings and join a mission-driven team committed to making AI accessible for everyone.

## High-Performance Solutions
For enterprises, we offer:
- **Compute Services**: Affordable GPU usage starting at $0.60/hour.
- **Enterprise Solutions**: Team collaboration tools with enterprise-grade security, support, and access controls starting at $20/user/month.

---

Let's build the future of AI together! For more information, visit our website at [Hugging Face](https://huggingface.co). Join our community today!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

## Exercise

1. Links 생성하기 위해 첫 번째 Model Call
    - One-shot prompting -> Multi-shot prompting 


2. Brochure를 만들기 위해 두 번째 Model Call

   - Brochure가 특정 포멧에 맞게 생성될 수 있도록 Prompt를 추가

  
3. Brochure를 다른 언어로 번역하도록 세 번째 Model Call

   - (1)과 (2)와 다른 모델을 사용한다.

In [49]:
# Step 1
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ],
    "links": [
        {"type": contents page, "url": https://full.url/contents},
        {"type": info page, "url": https://full.url/info}
    ]
}
"""

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    # \n을 구분자로 하여 하나의 문자열로 합침.
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        # openAI의 대답 형식을 지정할 수 있다.
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [50]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'company profile page',
   'url': 'https://huggingface.co/huggingface'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'models hub', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets hub', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces hub', 'url': 'https://huggingface.co/spaces'},
  {'type': 'product page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'product page', 'url': 'https://huggingface.co/chat'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'discord', 'url':

In [59]:
# Step 2
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    # GPT에서 관련 Link를 물어봄
    links = get_links(url)
    print("Found links:", links)
    # 각 Link에 해당되는 Website 객체의 내용을 Return
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += "The brochure must include the purpose, business direction, location, and contact information of this website."
    # 첫 번째 GPT Model을 통해 얻은 특정 Website에 대해 관련있는 Link의 Content를 전부 넘겨줌.
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

    return result

In [60]:
brochure = create_brochure("huggingface", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'company profile page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'brand assets page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'endpoints product page', 'url': 'https://endpoints.huggingface.co'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'models hub', 'url': 'https://huggingface.co/models'}, {'type': 'datasets hub', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces hub', 'url': 'https://huggingface.co/spaces'}, {'type': 'changelog', 'url': 'https://huggingface.co/changelog'}, {'type': 'community forum', 'u

# Hugging Face — The AI Community Building the Future

Hugging Face is the collaborative platform where the machine learning community creates, shares, and deploys models, datasets, and AI applications at global scale.

## Purpose
- Empower the world to build, share, and advance machine learning openly.
- Provide a home for ML collaboration across all modalities: text, image, video, audio, and 3D.

## What Hugging Face Offers
- Community Hub
  - 1M+ models, 250k+ datasets, and 400k+ applications (Spaces).
  - Discover, host, and collaborate on public models, datasets, and apps.
  - Build your ML profile and portfolio.
- Compute and Deployment
  - Inference Endpoints to deploy optimized model serving.
  - Scale Spaces apps to GPUs in a few clicks.
  - Pricing: GPU from $0.60/hour.
- Team & Enterprise
  - Advanced platform with security and governance.
  - Pricing: from $20/user/month.
  - Features: Single Sign-On, Regions, Priority Support, Audit Logs, Resource Groups, Private Datasets Viewer.

## Open Source Stack
Hugging Face builds foundational ML tooling together with the community:
- Transformers, Diffusers, Datasets, Tokenizers, PEFT, TRL, Accelerate
- Text Generation Inference (TGI), Safetensors, Hub Python Library, Transformers.js, smolagents

## Customers and Adoption
- Used by 50,000+ organizations.
- Examples: AI2, Meta, Amazon, Google, Intel, Microsoft, Grammarly, Writer.
- Active ecosystem with trending models and thousands of running Spaces.

## Culture and Community
- Open, collaborative, and community-first.
- Transparent development in open source, with contributions across research and production.
- Learn and share through Docs, Blog, Forum, and community apps (Spaces).

## Careers
- Explore roles on the Jobs page.
- Work on widely used open-source libraries and production AI infrastructure.
- Build tools that power research, startups, and enterprise AI teams globally.

## Business Direction
- Continue expanding the open-source ML ecosystem and community hub.
- Grow managed compute and enterprise offerings (secure deployment, governance, support).
- Enable faster, more reliable AI development across modalities and use cases.

## Location
- Global online platform serving users and organizations worldwide.
- Specific office locations are not listed on the provided pages.

## Contact and Resources
- Website: https://huggingface.co
- Enterprise and Pricing: available via the Pricing and Enterprise pages
- Documentation: Docs
- Community Help: Forum
- Service Health: Service Status
- Social: GitHub, Twitter, LinkedIn, Discord
- Company: About, Brand assets, Press, Jobs, Terms of Service, Privacy

Build, share, and deploy state-of-the-art AI with the world’s ML community at Hugging Face.

In [69]:
# Step 3

system_prompt = (
    "You are an English-to-Korean brochure translator. "
    "Translate the brochure into Korean and preserve the brochure's original format."
)

def get_user_prompt(brochure):
    user_prompt = "You should translate brochures into Korean"
    user_prompt += "Here are the contents of brochures"
    user_prompt += brochure

    return user_prompt

def translation(brochure):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(brochure)}
        ]
    )
    return response.choices[0].message.content

In [70]:
display(Markdown(translation(brochure)))

# Hugging Face — 미래를 만들어가는 AI 커뮤니티

Hugging Face는 머신러닝 커뮤니티가 전 세계 규모로 모델, 데이터셋, AI 애플리케이션을 만들고, 공유하며, 배포하는 협업 플랫폼입니다.

## 목적
- 전 세계가 머신러닝을 개방적으로 구축·공유·발전시키도록 돕습니다.
- 텍스트, 이미지, 비디오, 오디오, 3D 등 모든 모달리티 전반의 ML 협업을 위한 터전을 제공합니다.

## Hugging Face가 제공하는 것
- 커뮤니티 허브
  - 1M+ 모델, 250k+ 데이터셋, 400k+ 애플리케이션(Spaces).
  - 공개 모델, 데이터셋, 앱을 탐색하고 호스팅하며 협업하세요.
  - 나만의 ML 프로필과 포트폴리오를 구축하세요.
- 컴퓨팅 및 배포
  - 최적화된 모델 서빙을 배포하기 위한 Inference Endpoints.
  - 몇 번의 클릭만으로 Spaces 앱을 GPU로 확장.
  - 요금: GPU 시간당 $0.60부터.
- 팀 및 엔터프라이즈
  - 보안과 거버넌스를 갖춘 고급 플랫폼.
  - 요금: 사용자당 월 $20부터.
  - 기능: 싱글 사인온(SSO), 리전, 우선 지원, 감사 로그, 리소스 그룹, 비공개 데이터셋 뷰어.

## 오픈소스 스택
Hugging Face는 커뮤니티와 함께 핵심 ML 도구를 구축합니다:
- Transformers, Diffusers, Datasets, Tokenizers, PEFT, TRL, Accelerate
- Text Generation Inference (TGI), Safetensors, Hub Python Library, Transformers.js, smolagents

## 고객 및 도입 현황
- 50,000+개 조직이 사용 중.
- 예시: AI2, Meta, Amazon, Google, Intel, Microsoft, Grammarly, Writer.
- 인기 모델과 수천 개의 실행 중인 Spaces가 있는 활발한 생태계.

## 문화와 커뮤니티
- 개방적이고 협업적이며 커뮤니티 우선.
- 연구부터 프로덕션까지 기여가 이루어지는 오픈소스에서의 투명한 개발.
- Docs, Blog, Forum, 커뮤니티 앱(Spaces)을 통해 배우고 공유하세요.

## 채용
- Jobs 페이지에서 역할을 확인하세요.
- 전 세계적으로 사용되는 오픈소스 라이브러리와 프로덕션 AI 인프라를 함께 구축하세요.
- 전 세계 연구, 스타트업, 엔터프라이즈 AI 팀을 지원하는 도구를 만드세요.

## 비즈니스 방향
- 오픈소스 ML 생태계와 커뮤니티 허브를 지속적으로 확장.
- 관리형 컴퓨팅 및 엔터프라이즈 제공(보안 배포, 거버넌스, 지원)을 확대.
- 다양한 모달리티와 유스케이스 전반에서 더 빠르고 신뢰할 수 있는 AI 개발을 가능하게 함.

## 위치
- 전 세계 사용자와 조직을 지원하는 글로벌 온라인 플랫폼.
- 제공된 페이지에는 특정 오피스 위치가 기재되어 있지 않습니다.

## 문의 및 리소스
- Website: https://huggingface.co
- Enterprise and Pricing: Pricing 및 Enterprise 페이지에서 확인 가능
- Documentation: Docs
- Community Help: Forum
- Service Health: Service Status
- Social: GitHub, Twitter, LinkedIn, Discord
- Company: About, Brand assets, Press, Jobs, Terms of Service, Privacy

전 세계 ML 커뮤니티와 함께 Hugging Face에서 최첨단 AI를 구축하고, 공유하며, 배포하세요.